In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

홈 어드밴티지는 존재할까?

In [4]:
from pathlib import Path

BASE_DIR = Path.cwd()
if BASE_DIR.name == "notebooks":
    BASE_DIR = BASE_DIR.parent

In [5]:
df = pd.read_csv(BASE_DIR/"data"/"champions_league_matches.csv")
df

,date,home_team,away_team,score,venue,referee,home_possession,away_possession,home_shots_on_target,away_shots_on_target,home_saves,away_saves,home_shots_on_target_pct,away_shots_on_target_pct,home_saves_pct,away_saves_pct,result,winner
0,2025-09-16,PSV,Union SG,1–3,Philips Stadion,Anthony Taylor,63%,37%,3 of 10,8 of 18,4 of 8,2 of 3,30.0,44.4,50.0,66.7,Away Win,Union SG
1,2025-09-16,Athletic Club,Arsenal,0–2,San Mamés,Donatas Rumšas,38%,62%,2 of 11,6 of 11,4 of 6,2 of 2,18.2,54.5,66.7,100.0,Away Win,Arsenal
2,2025-09-16,Tottenham Hotspur,Villarreal,1–0,Tottenham Hotspur Stadium,Rade Obrenović,58%,42%,1 of 9,0 of 10,0 of 0,1 of 1,11.1,0.0,NaN,100.0,Home Win,Tottenham Hotspur
3,2025-09-16,Benfica,Qarabağ,2–3,Estádio do Sport Lisboa e Benfica,Erik Lambrechts,53%,47%,3 of 14,5 of 10,2 of 5,1 of 3,21.4,50.0,40.0,33.3,Away Win,Qarabağ
4,2025-09-16,Juventus,Dortmund,4–4,Allianz Stadium,François Letexier,52%,48%,7 of 19,5 of 10,1 of 5,3 of 7,36.8,50.0,20.0,42.9,Draw,Draw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2026-01-28,Barcelona,FC Copenhagen,4–1,Spotify Camp Nou,Benoît Bastien,77%,24%,13 of 28,1 of 6,0 of 1,8 of 13,46.4,16.7,0.0,61.5,Home Win,Barcelona
147,2026-01-28,Leverkusen,Villarreal,3–0,BayArena,Espen Eskås,65%,35%,7 of 15,2 of 5,2 of 2,4 of 7,46.7,40.0,100.0,57.1,Home Win,Leverkusen
148,2026-01-28,Paris Saint-Germain,Newcastle United,1–1,Parc des Princes,Slavko Vinčič,67%,33%,7 of 25,4 of 10,3 of 4,5 of 7,28.0,40.0,75.0,71.4,Draw,Draw
149,2026-01-28,Napoli,Chelsea,2–3,Stadio Diego Armando Maradona,Clément Turpin,46%,54%,5 of 8,5 of 11,2 of 5,3 of 5,62.5,45.5,40.0,60.0,Away Win,Chelsea


In [6]:
df['score_clean'] = (
    df['score']
    .str.replace('–', '-', regex=False)
    .str.replace('—', '-', regex=False)
    .str.strip()
)

#노란박스 '-' -> Alt+0150 (En dash : 범수, 점수, 대결)
#긴 막대기 '—' -> Alt+0151 (Em dash : 문장 끊기)

In [7]:
df[['home_goals','away_goals']] = (
    df['score_clean']
    .str.split('-', expand=True)
    .apply(pd.to_numeric, errors='coerce')
)

In [8]:
def get_team_result(result, side):
    if result == "Draw":
        return "D"
    if side == "home":
        return "W" if result == "Home Win" else "L"
    if side == "away":
        return "W" if result == "Away Win" else "L"

In [9]:
df['home_result'] = df['result'].apply(lambda x: get_team_result(x, 'home'))
df['away_result'] = df['result'].apply(lambda x: get_team_result(x, 'away'))

In [10]:
for col in ['home_possession','away_possession']:
    df[col] = df[col].str.replace("%","").astype(float)

In [11]:
df

,date,home_team,away_team,score,venue,referee,home_possession,away_possession,home_shots_on_target,away_shots_on_target,...,away_shots_on_target_pct,home_saves_pct,away_saves_pct,result,winner,score_clean,home_goals,away_goals,home_result,away_result
0,2025-09-16,PSV,Union SG,1–3,Philips Stadion,Anthony Taylor,63.0,37.0,3 of 10,8 of 18,...,44.4,50.0,66.7,Away Win,Union SG,1-3,1.0,3.0,L,W
1,2025-09-16,Athletic Club,Arsenal,0–2,San Mamés,Donatas Rumšas,38.0,62.0,2 of 11,6 of 11,...,54.5,66.7,100.0,Away Win,Arsenal,0-2,0.0,2.0,L,W
2,2025-09-16,Tottenham Hotspur,Villarreal,1–0,Tottenham Hotspur Stadium,Rade Obrenović,58.0,42.0,1 of 9,0 of 10,...,0.0,NaN,100.0,Home Win,Tottenham Hotspur,1-0,1.0,0.0,W,L
3,2025-09-16,Benfica,Qarabağ,2–3,Estádio do Sport Lisboa e Benfica,Erik Lambrechts,53.0,47.0,3 of 14,5 of 10,...,50.0,40.0,33.3,Away Win,Qarabağ,2-3,2.0,3.0,L,W
4,2025-09-16,Juventus,Dortmund,4–4,Allianz Stadium,François Letexier,52.0,48.0,7 of 19,5 of 10,...,50.0,20.0,42.9,Draw,Draw,4-4,4.0,4.0,D,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2026-01-28,Barcelona,FC Copenhagen,4–1,Spotify Camp Nou,Benoît Bastien,77.0,24.0,13 of 28,1 of 6,...,16.7,0.0,61.5,Home Win,Barcelona,4-1,4.0,1.0,W,L
147,2026-01-28,Leverkusen,Villarreal,3–0,BayArena,Espen Eskås,65.0,35.0,7 of 15,2 of 5,...,40.0,100.0,57.1,Home Win,Leverkusen,3-0,3.0,0.0,W,L
148,2026-01-28,Paris Saint-Germain,Newcastle United,1–1,Parc des Princes,Slavko Vinčič,67.0,33.0,7 of 25,4 of 10,...,40.0,75.0,71.4,Draw,Draw,1-1,1.0,1.0,D,D
149,2026-01-28,Napoli,Chelsea,2–3,Stadio Diego Armando Maradona,Clément Turpin,46.0,54.0,5 of 8,5 of 11,...,45.5,40.0,60.0,Away Win,Chelsea,2-3,2.0,3.0,L,W


In [12]:
df.columns
#shots_on_target : 유효슈팅
#shots_on_target_pct : 전체슈팅 중 유효슈팅 비율
#saves_pct : 선방률

Index(['date', 'home_team', 'away_team', 'score', 'venue', 'referee',
       'home_possession', 'away_possession', 'home_shots_on_target',
       'away_shots_on_target', 'home_saves', 'away_saves',
       'home_shots_on_target_pct', 'away_shots_on_target_pct',
       'home_saves_pct', 'away_saves_pct', 'result', 'winner', 'score_clean',
       'home_goals', 'away_goals', 'home_result', 'away_result'],
      dtype='object')

In [13]:
home_cols = [c for c in df.columns if c.startswith("home_")]
df_home = df[home_cols]
df_home

,home_team,home_possession,home_shots_on_target,home_saves,home_shots_on_target_pct,home_saves_pct,home_goals,home_result
0,PSV,63.0,3 of 10,4 of 8,30.0,50.0,1.0,L
1,Athletic Club,38.0,2 of 11,4 of 6,18.2,66.7,0.0,L
2,Tottenham Hotspur,58.0,1 of 9,0 of 0,11.1,NaN,1.0,W
3,Benfica,53.0,3 of 14,2 of 5,21.4,40.0,2.0,L
4,Juventus,52.0,7 of 19,1 of 5,36.8,20.0,4.0,D
...,...,...,...,...,...,...,...,...
146,Barcelona,77.0,13 of 28,0 of 1,46.4,0.0,4.0,W
147,Leverkusen,65.0,7 of 15,2 of 2,46.7,100.0,3.0,W
148,Paris Saint-Germain,67.0,7 of 25,3 of 4,28.0,75.0,1.0,D
149,Napoli,46.0,5 of 8,2 of 5,62.5,40.0,2.0,L


In [14]:
away_cols = [c for c in df.columns if c.startswith("away_")]
df_away = df[away_cols]
df_away

,away_team,away_possession,away_shots_on_target,away_saves,away_shots_on_target_pct,away_saves_pct,away_goals,away_result
0,Union SG,37.0,8 of 18,2 of 3,44.4,66.7,3.0,W
1,Arsenal,62.0,6 of 11,2 of 2,54.5,100.0,2.0,W
2,Villarreal,42.0,0 of 10,1 of 1,0.0,100.0,0.0,L
3,Qarabağ,47.0,5 of 10,1 of 3,50.0,33.3,3.0,W
4,Dortmund,48.0,5 of 10,3 of 7,50.0,42.9,4.0,D
...,...,...,...,...,...,...,...,...
146,FC Copenhagen,24.0,1 of 6,8 of 13,16.7,61.5,1.0,L
147,Villarreal,35.0,2 of 5,4 of 7,40.0,57.1,0.0,L
148,Newcastle United,33.0,4 of 10,5 of 7,40.0,71.4,1.0,D
149,Chelsea,54.0,5 of 11,3 of 5,45.5,60.0,3.0,W


In [17]:
away_team_stats = (
    df_away
    .groupby(["away_team","away_result"], as_index=False)
    .agg(
        matches=("away_team", "count"),
        avg_possession=("away_possession", "mean"),
        avg_sot=("away_shots_on_target_pct", "mean"),
        avg_save=("away_saves_pct","mean"),
        avg_goals_for=("away_goals", "mean"),
    )
)
away_team_stats

,away_team,away_result,matches,avg_possession,avg_sot,avg_save,avg_goals_for
0,Ajax,L,2,43.00,32.700,41.650,0.50
1,Ajax,W,2,57.50,36.650,70.850,3.00
2,Arsenal,W,4,54.75,50.475,93.750,2.75
3,Atalanta,L,2,48.50,22.000,55.750,0.00
4,Atalanta,W,2,50.50,35.400,100.000,2.00
...,...,...,...,...,...,...,...
79,Tottenham Hotspur,L,1,33.00,45.500,16.700,3.00
80,Tottenham Hotspur,W,1,55.00,40.000,100.000,2.00
81,Union SG,L,2,30.00,28.750,50.000,0.50
82,Union SG,W,2,35.50,37.800,66.700,2.00


In [16]:
home_team_stats = (
    df_home
    .groupby(["home_team","home_result"], as_index=False)
    .agg(
        matches=("home_team", "count"),
        avg_possession=("home_possession", "mean"),
        avg_sot=("home_shots_on_target_pct", "mean"),
        avg_save=("home_saves_pct","mean"),
        avg_goals_for=("home_goals", "mean"),
    )
)
home_team_stats

,home_team,home_result,matches,avg_possession,avg_sot,avg_save,avg_goals_for
0,Ajax,L,4,57.500000,33.225000,53.850000,0.250000
1,Arsenal,W,4,54.500000,46.000000,62.500000,3.000000
2,Atalanta,D,1,56.000000,22.700000,100.000000,0.000000
3,Atalanta,L,1,70.000000,28.600000,0.000000,2.000000
4,Atalanta,W,2,48.500000,21.800000,65.000000,2.000000
...,...,...,...,...,...,...,...
74,Tottenham Hotspur,W,4,56.500000,36.925000,100.000000,2.500000
75,Union SG,L,3,38.000000,50.100000,38.733333,0.666667
76,Union SG,W,1,36.000000,50.000000,100.000000,1.000000
77,Villarreal,D,1,44.000000,35.300000,50.000000,2.000000
